* 使用BERT 取得Item的Embedding
* 使用BERT 把 User買過的所有Item加總起來取平均，作為User的Embedding
* 使用K-means對768維的向量做分群
* 用t-SNE將768維降到2維並畫在圖上

In [1]:
%config Completer.use_jedi = False

In [2]:
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import matplotlib.cm as cm 

In [3]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report

dtype = torch.float32
device = torch.device('cuda:0')
device

device(type='cuda', index=0)

In [29]:
txn

,訂單編號,下單日期,時間戳記,psid,asid,user_id,channel_name,商品id,商品名稱,樣式,...,付款單號,專屬折扣,運費,收款金額,付款方式,運送方式,寄送地址,場次,date,item_name
0,BD220301-86612477,2022-03-01,00:04:56,2144742712282981,2912039312151026,10891,邦成-自倉（C倉出貨付款全開）,6492974,（免運）(闆)公主 0228 A01-(限+1)(免運)(3件)100%石墨烯遠紅外線棉被(...,NaN,...,PA220301-15817276,0,0,14044,LINE Pay,一般宅配,桃園市蘆竹區大興十四街77號,20220228(公主x深-重播0221 888C倉晚上A1)-原,2022-03-01,公主 0228 A01-(限+1)(免運)(3件)100%石墨烯遠紅外線棉被(180X210...
1,BD220301-86612495,2022-03-01,00:05:00,4880729621967911,1681086192072457,10891,邦成-自倉（C倉出貨付款全開）,6493322,大師兄 0228 A01-金屬玻璃調味瓶*1,NaN,...,PA220301-15831018,0,150,2048,貨到付款,一般宅配,新北市樹林區俊英街155巷43弄15號4樓之3,20220228(大師兄xC倉深夜場1)-儒/川/駿,2022-03-01,大師兄 0228 A01-金屬玻璃調味瓶*1
2,BD220301-86612507,2022-03-01,00:05:00,3861530257224160,2836025813335298,10891,邦成-自倉（C倉出貨付款全開）,6493322,大師兄 0228 A01-金屬玻璃調味瓶*1,NaN,...,PA220301-15815246,0,0,595,自取,一般宅配,自取點-B倉,20220228(大師兄xC倉深夜場1)-儒/川/駿,2022-03-01,大師兄 0228 A01-金屬玻璃調味瓶*1
3,BD220301-86612510,2022-03-01,00:05:00,3521679651227760,3120032084685049,10891,邦成-自倉（C倉出貨付款全開）,6493322,大師兄 0228 A01-金屬玻璃調味瓶*1,NaN,...,PA220303-15883127,0,0,14664,匯款,一般宅配,南投縣竹山鎮大同街111號,20220228(大師兄xC倉深夜場1)-儒/川/駿,2022-03-03,大師兄 0228 A01-金屬玻璃調味瓶*1
4,BD220301-86612516,2022-03-01,00:05:02,2542831359072575,3383335041684491,10891,邦成-自倉（C倉出貨付款全開）,6478836,888 0227 A18-(5盒)BRE 療肺草暢呼吸飲(8包/盒)*5,NaN,...,PA220303-15877459,0,0,3866,自取,一般宅配,自取點-B倉,20220227(888xC倉下午場1)-玲/哲/砡,2022-03-03,888 0227 A18-(5盒)BRE 療肺草暢呼吸飲(8包/盒)*5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5788747,BD220430-96897943,2022-04-30,14:57:03,NaN,2246133855485630,4793,279-🛒現貨直播車,6756827,【商城】Goongbe宮中秘策48小時 寶寶保濕霜30ml(202203170001),NaN,...,PA220430-17836184,0,0,215,自取,一般宅配,自取點-總店,NaN,2022-04-30,Goongbe宮中秘策48小時 寶寶保濕霜30ml(202203170001)
5788748,BD220430-96898373,2022-04-30,15:01:56,NaN,2246133855485630,4793,279-🛒現貨直播車,7277971,【商城】Dripoドリポ牧場 **印度** 紅茶牛乳(1條),NaN,...,PA220430-17836184,0,0,215,自取,一般宅配,自取點-總店,NaN,2022-04-30,Dripoドリポ牧場 **印度** 紅茶牛乳(1條)
5788751,BD220430-96908225,2022-04-30,16:13:23,NaN,2717578198260964,4793,279-🛒現貨直播車,6556105,【商城】亞大T8銀耳 冰糖銀耳羹(一瓶)(202112270007),NaN,...,PA220501-17866137,0,60,2520,貨到付款,一般宅配,桃園市楊梅區中山北路1段360號,NaN,2022-05-01,亞大T8銀耳 冰糖銀耳羹(一瓶)(202112270007)
5788752,BD220430-96910480,2022-04-30,16:32:37,NaN,2896144450612065,4793,279-🛒現貨直播車,7145579,【商城】Binaca舌苔刷(一支)(202204110007),NaN,...,PA220430-17838652,0,70,715,超取付,7-11C2C,7-ELEVEN 遠德門市（地址：台南市新化區北勢里中正路1211號 / 電話：--）,NaN,2022-04-30,Binaca舌苔刷(一支)(202204110007)


In [31]:
txn = pd.read_pickle('data/new_txn.pkl')
txn = txn[~txn['場次'].isna()]
txn = txn.reset_index(drop=True)
txn

,訂單編號,下單日期,時間戳記,psid,asid,user_id,channel_name,商品id,商品名稱,樣式,...,付款單號,專屬折扣,運費,收款金額,付款方式,運送方式,寄送地址,場次,date,item_name
0,BD220301-86612477,2022-03-01,00:04:56,2144742712282981,2912039312151026,10891,邦成-自倉（C倉出貨付款全開）,6492974,（免運）(闆)公主 0228 A01-(限+1)(免運)(3件)100%石墨烯遠紅外線棉被(...,NaN,...,PA220301-15817276,0,0,14044,LINE Pay,一般宅配,桃園市蘆竹區大興十四街77號,20220228(公主x深-重播0221 888C倉晚上A1)-原,2022-03-01,公主 0228 A01-(限+1)(免運)(3件)100%石墨烯遠紅外線棉被(180X210...
1,BD220301-86612495,2022-03-01,00:05:00,4880729621967911,1681086192072457,10891,邦成-自倉（C倉出貨付款全開）,6493322,大師兄 0228 A01-金屬玻璃調味瓶*1,NaN,...,PA220301-15831018,0,150,2048,貨到付款,一般宅配,新北市樹林區俊英街155巷43弄15號4樓之3,20220228(大師兄xC倉深夜場1)-儒/川/駿,2022-03-01,大師兄 0228 A01-金屬玻璃調味瓶*1
2,BD220301-86612507,2022-03-01,00:05:00,3861530257224160,2836025813335298,10891,邦成-自倉（C倉出貨付款全開）,6493322,大師兄 0228 A01-金屬玻璃調味瓶*1,NaN,...,PA220301-15815246,0,0,595,自取,一般宅配,自取點-B倉,20220228(大師兄xC倉深夜場1)-儒/川/駿,2022-03-01,大師兄 0228 A01-金屬玻璃調味瓶*1
3,BD220301-86612510,2022-03-01,00:05:00,3521679651227760,3120032084685049,10891,邦成-自倉（C倉出貨付款全開）,6493322,大師兄 0228 A01-金屬玻璃調味瓶*1,NaN,...,PA220303-15883127,0,0,14664,匯款,一般宅配,南投縣竹山鎮大同街111號,20220228(大師兄xC倉深夜場1)-儒/川/駿,2022-03-03,大師兄 0228 A01-金屬玻璃調味瓶*1
4,BD220301-86612516,2022-03-01,00:05:02,2542831359072575,3383335041684491,10891,邦成-自倉（C倉出貨付款全開）,6478836,888 0227 A18-(5盒)BRE 療肺草暢呼吸飲(8包/盒)*5,NaN,...,PA220303-15877459,0,0,3866,自取,一般宅配,自取點-B倉,20220227(888xC倉下午場1)-玲/哲/砡,2022-03-03,888 0227 A18-(5盒)BRE 療肺草暢呼吸飲(8包/盒)*5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5442250,BD220428-96524701,2022-04-28,01:35:45,7154387501298874,3521523544542060,4793,279-🛒現貨直播車,7258649,第 044 標 (4/27)A5 麻辣臭豆腐450g,120,...,PA220428-17757414,0,0,3024,超取付,7-11C2C,7-ELEVEN 善文門市（地址：台南市善化區中山路518號 / 電話：--）,20220428.0,2022-04-28,第 044 標 (4/27)A5 麻辣臭豆腐450g
5442251,BD220428-96573804,2022-04-28,16:32:46,NaN,107685158351211,4793,279-🛒現貨直播車,7257663,第 022 標 (4/27)BOSS 側邊線條LOGO 棉長褲,32,...,PA220430-17834886,0,70,2655,超取付,7-11C2C,7-ELEVEN 嘉太門市（地址：嘉義縣太保市北港路二段190號 / 電話：--）,20220427.0,2022-04-30,第 022 標 (4/27)BOSS 側邊線條LOGO 棉長褲
5442252,BD220428-96575102,2022-04-28,16:42:27,NaN,4059356824077353,4793,279-🛒現貨直播車,7250821,第 007 標 (4/27)德國 高劑量1000單位維他命C發泡錠(20錠),180,...,PA220501-17853309,0,40,1499,信用卡,一般宅配,台北市士林區中山北路五段700號1樓,20220427.0,2022-05-01,第 007 標 (4/27)德國 高劑量1000單位維他命C發泡錠(20錠)
5442253,BD220428-96575527,2022-04-28,16:45:12,NaN,2246133855485630,4793,279-🛒現貨直播車,7250826,@第 029 標 (4/27) TOP 脆米威化餅 (印尼的雷神巧克力)(2隻 隨機),25,...,PA220428-17774899,0,0,375,自取,一般宅配,自取點-總店,20220427.0,2022-04-28,@第 029 標 (4/27) TOP 脆米威化餅 (印尼的雷神巧克力)(2隻 隨機)


In [36]:
txn['stream'] = txn['user_id'].astype(str) + '@' + txn['場次'].astype(str)

In [38]:
iname2embedding = np.load('data/jambo_iname2embedding.npy', allow_pickle=True)[None][0]
iname2embedding['ORIGINS品木宣言靈芝水200ml']

array([ 5.80445051e-01,  3.84010494e-01, -1.96773812e-01, -4.09691483e-01,
        1.02684826e-01, -4.08415645e-01,  5.23488998e-01,  2.04683110e-01,
       -4.08595294e-01, -2.07563460e-01, -2.67374068e-01, -5.21873593e-01,
       -5.04533947e-01,  1.88132003e-01,  1.67892486e-01, -3.85657325e-02,
        6.85403585e-01, -3.23914774e-02,  3.02290827e-01, -5.12680113e-02,
        2.65854269e-01, -3.90807241e-02, -3.11178982e-01,  4.32411619e-02,
       -2.84091055e-01, -1.39492638e-02,  2.75702596e-01, -1.27285849e-02,
       -4.71977204e-01,  3.51862073e-01, -3.26199055e-01,  3.57699156e-01,
       -5.45297027e-01,  2.47347221e-01,  1.40018597e-01,  5.38950682e-01,
       -1.90762460e-01, -3.46850306e-01, -1.33837700e-01, -6.82761669e-01,
       -2.63975352e-01,  4.52496827e-01,  1.06706411e-01, -8.19984227e-02,
        4.63603348e-01,  4.10682648e-01, -7.67347440e-02,  3.96633834e-01,
       -1.89128429e-01,  3.06273669e-01,  1.63638398e-01,  1.45282373e-01,
       -1.75846424e-02, -

In [39]:
item_names = txn['item_name'].unique().tolist()
print(len(item_names))
item_names[:10]

158539


['公主 0228 A01-(限+1)(免運)(3件)100%石墨烯遠紅外線棉被(180X210)*3-價錢後台看',
 '大師兄 0228 A01-金屬玻璃調味瓶*1',
 '888 0227 A18-(5盒)BRE 療肺草暢呼吸飲(8包/盒)*5',
 '大師兄 0228 A06-秋紅饗胡椒豬肚雞 (450g)*1',
 '888 0227 A17-(5罐)BRE台農膠原蛋白肽粉(150g/罐)*5-結單缺貨刪單不通知',
 '888 0228 A03-洗衣機槽專用強力清潔錠(16入/盒)*1',
 '888 0227 A04-FOUR SEASONS擴香瓶200ML(隨機)*1-結單缺貨刪單不通知',
 '公主 0228 A09-(8盒)BRE阿育吠陀 七脈輪淨化植萃膠囊(30顆/盒)*8',
 '大師兄 0228 A02-25*25 加大 珊瑚絨抹布(10入)*1',
 '畢哥 0228 A05-(6件組)PRINCESS PARTY 超皙錠(30粒/盒)*5+Mixzuzu 美容膠原蛋白莓妍飲190ml(24瓶/箱)－草莓奶昔*1--結單缺貨刪單不通知']

In [40]:
item_name_max_len = max([len(i) for i in item_names])
item_name_max_len

101

### 使用BERT取得商品名稱的向量

In [8]:
# from transformers import (
#   BertTokenizerFast,
#   AutoModel,
# )
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-chinese')

# # model = AutoModel.from_pretrained('ckiplab/bert-base-chinese-ws')
# model_BERT = AutoModel.from_pretrained('ckiplab/bert-base-chinese')
# model_BERT.to(device)

In [9]:
# %%time
# pad_token_item_names = tokenizer.batch_encode_plus(
#     item_names, max_length=item_name_max_len, truncation=True, padding=True)['input_ids']
# pad_token_item_names = torch.tensor(pad_token_item_names, device=device)
# pad_token_item_names

In [10]:
# class BertTokenDataset(Dataset):
#     def __init__(self, tokens):
#         self.tokens = tokens
        
#     def __len__(self):
#         return len(self.tokens)

#     def __getitem__(self, idx):
#         return self.tokens[idx]

# token_dataset = BertTokenDataset(pad_token_item_names)
# token_dataloader = DataLoader(token_dataset, shuffle=False, batch_size=32)

In [11]:
# # 一次跑全部的 item 會 CUDA out of memory
# # item_name_embedding = model(pad_token_item_names)

# item_name_embedding = []
# for token in tqdm(token_dataloader):
#     embedding = model_BERT(token).pooler_output
#     embedding = embedding.tolist()
#     item_name_embedding.extend(embedding)
# item_name_embedding = np.array(item_name_embedding)
# item_name_embedding.shape

In [12]:
# iname2embedding = {n:emb for n, emb in zip(item_names, item_name_embedding)}

In [41]:
iname2embedding_index = {name:i for i, name in enumerate(item_names)}
iname2embedding_index['ORIGINS品木宣言靈芝水200ml']

8967

In [42]:
item_name_embedding = []
for iname in tqdm(item_names):
    item_name_embedding.append(iname2embedding[iname])
    
item_name_embedding = np.array(item_name_embedding)
item_name_embedding.shape

  0%|          | 0/158539 [00:00<?, ?it/s]

(158539, 768)

In [72]:
item_name_embedding[[1, 100, 2500, 9987]].shape

(4, 768)

In [43]:
user_stream_buy_list = txn.groupby(['asid', 'stream'])['item_name'].apply(list)
user_stream_buy_list

asid                               stream                                     
1000057940522534                   10891@20210625(舊三郎xC倉晚上場1)-嘻/鈞/砡/俞             [舊三郎 0625 A08-美國DORCO 柔軟細刷毛牙刷 (12PC/盒)*1, 舊三郎 ...
                                   10891@20210626(舊三郎xC倉晚上場1)-儒/砡/哲               [舊三郎 0626 A13-56吋雨傘(顏色隨機)*1, 舊三郎 0626 A14-萬向調節...
                                   10891@20210720(舊三郎x晚-重播0719 888xC倉晚上D1)-赫      [舊三郎 0720 666-(限+1)0719賺到福箱(不接受不滿意商品退換貨), 舊三郎 ...
                                   10891@20210724(舊三郎xC倉晚上場1)-儒/駿/瑞               [舊三郎 0724 A18-(10組)怡百麗牆面修補膏 (250g)+嘴+刮刀*10, 舊三...
                                   10891@20210822(舊三郎x午-重播0821 888x四季天韻下午B1)-志    [舊三郎 0822 A04-(3罐)斯諾爾止鼾噴霧25ml*3, 舊三郎 0822 康普-(...
                                                                                                        ...                        
Uffb50137571903b41efcbb49829729a7  10891@20210319(舊三郎x午-重播0315公主x888C倉外下午B)-赫     [舊三郎 0319 A01-(4罐)七星級酒店擴香 120ml (味道隨機)*4--超收刪單.

In [57]:
user_stream_buy_list['1000057940522534', '10891@20210625(舊三郎xC倉晚上場1)-嘻/鈞/砡/俞']

['舊三郎 0625 A08-美國DORCO 柔軟細刷毛牙刷 (12PC/盒)*1',
 '舊三郎 0625 A09-(8顆)竹炭美顏皂*8',
 '舊三郎 0625 A31-超微氣泡美容儀(SX-128A)*1',
 '舊三郎 0625 A33-多用途LED充電防潑水吊掛燈*1',
 '舊三郎 0625 A36-16吋電風扇(會感冒的扇)(ES-1886)*1']

In [56]:
user_stream_buy_list.index[0]

('1000057940522534', '10891@20210625(舊三郎xC倉晚上場1)-嘻/鈞/砡/俞')

In [58]:
for asid, stream in tqdm(user_stream_buy_list.index):
    buylist = user_stream_buy_list[asid, stream]
    print(asid, stream)
    print(buylist)
    break

  0%|          | 0/1568307 [00:00<?, ?it/s]

1000057940522534 10891@20210625(舊三郎xC倉晚上場1)-嘻/鈞/砡/俞
['舊三郎 0625 A08-美國DORCO 柔軟細刷毛牙刷 (12PC/盒)*1', '舊三郎 0625 A09-(8顆)竹炭美顏皂*8', '舊三郎 0625 A31-超微氣泡美容儀(SX-128A)*1', '舊三郎 0625 A33-多用途LED充電防潑水吊掛燈*1', '舊三郎 0625 A36-16吋電風扇(會感冒的扇)(ES-1886)*1']


## 把User在某場直播買過的商品的BERT Embedding 加總平均
### 第一種作法

In [59]:
user_embedding = dict()
for asid, stream in tqdm(user_stream_buy_list.index):
    buylist = user_stream_buy_list[asid, stream]
    user_emb = np.zeros(768)
    for item in buylist:
        user_emb = np.vstack((user_emb, iname2embedding[item]))
    user_emb = user_emb[1:, :]
    
    user_emb = user_emb.mean(axis=0)
    user_embedding[(asid, stream)] = user_emb

  0%|          | 0/1568307 [00:00<?, ?it/s]

KeyboardInterrupt: 

### 第二種作法

In [65]:
user_embedding_2 = dict()
for asid, stream in tqdm(user_stream_buy_list.index):
    buylist = user_stream_buy_list[asid, stream]
    
    all_buy_item_index = [iname2embedding_index[item] for item in buylist]
    user_emb = item_name_embedding[all_buy_item_index].mean(axis=0)
    user_embedding_2[(asid, stream)] = user_emb

  0%|          | 0/1568307 [00:00<?, ?it/s]

In [69]:
test_key = list(user_embedding.keys())[2]
user_embedding_2[test_key] == user_embedding[test_key]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,